In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
os.chdir("../input")
os.listdir()

## **Data Read**

In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv("../input/laptop-price/laptop_price.csv",encoding='latin-1')

In [1]:
df.info()

In [1]:
df.isnull().sum()

In [1]:
df.head(10)

## **DATA CLEANING AND PREPERATİON**

In [1]:
df["Ram"] = df["Ram"].str.replace('GB', '') ## remove 'GB'

In [1]:
df["Weight"] = df["Weight"].str.replace('kg', '') ## remove 'kg'

In [1]:
df['Memory'] = df['Memory'].astype(str).replace('\.0', '', regex=True) ## '.0' part remove ex; '1.0'---> '1' 

In [1]:
df["Memory"] = df["Memory"].str.replace('GB', '') ## remove 'GB'

In [1]:
df["Memory"] = df["Memory"].str.replace('TB', '000') # convert to TB to GB values ex; 1TB=1000GB

In [1]:
new2 = df["Memory"].str.split("+", n = 1, expand = True) # seperate according to '+'

In [1]:
# making separate first name column from new data frame 
df["first"]= new2[0]
df["first"]=df["first"].str.strip() # First part of Memory

In [1]:
# making separate last name column from new data frame 
df["second"]= new2[1] # Second part of Memory

In [1]:
### Categorization of first part and second part of Memory
# Categorization of first part of Memory

df["Layer1HDD"] = df["first"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer1SSD"] = df["first"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer1Hybrid"] = df["first"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer1Flash_Storage"] = df["first"].apply(lambda x: 1 if "Flash Storage" in x else 0)

In [1]:
# Data Cleaning for first part unwanted charecters 
df['first'] = df['first'].str.replace(r'\D', '')

In [1]:
# For Consisting of one part Memory fill the Na values with 0
df["second"].fillna("0", inplace = True)

In [1]:
# Categorization of second part of Memory
df["Layer2HDD"] = df["second"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer2SSD"] = df["second"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer2Hybrid"] = df["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)

In [1]:
## Data Cleaning for second part unwanted charecters 
df['second'] = df['second'].str.replace(r'\D', '')

In [1]:
df.info()

In [1]:
# Convert to integer first and second parts 
df["first"] = df["first"].astype(int)
df["second"] = df["second"].astype(int)

In [1]:
df.info()

In [1]:
# Calculation of Total Memory
df["Total_Memory"]=(df["first"]*(df["Layer1HDD"]+df["Layer1SSD"]+df["Layer1Hybrid"]+df["Layer1Flash_Storage"])+df["second"]*(df["Layer2HDD"]+df["Layer2SSD"]+df["Layer2Hybrid"]+df["Layer2Flash_Storage"]))
df["Memory"]=df["Total_Memory"]

In [1]:
# Calculation of Category HDD,SSD,Hybrid,Flash Storage
df["HDD"]=(df["first"]*df["Layer1HDD"]+df["second"]*df["Layer2HDD"])
df["SSD"]=(df["first"]*df["Layer1SSD"]+df["second"]*df["Layer2SSD"])
df["Hybrid"]=(df["first"]*df["Layer1Hybrid"]+df["second"]*df["Layer2Hybrid"])
df["Flash_Storage"]=(df["first"]*df["Layer1Flash_Storage"]+df["second"]*df["Layer2Flash_Storage"])

In [1]:
# Seperate Screen Resolution Part according to 'x' character
# new data frame with split value columns 
new = df["ScreenResolution"].str.split("x", n = 1, expand = True) 

In [1]:
# X resolution 
# making separate first name column from new data frame 
df["X_res"]= new[0] 

In [1]:
# Y resolution 
# making separate last name column from new data frame 
df["Y_res"]= new[1] 

In [1]:
df["Y_res"]=pd.to_numeric(df["Y_res"])

In [1]:
df["Y_res"] = df["Y_res"].astype(float)

In [1]:
# Data Cleaning for X_res part unwanted charecters 
df["X_res"]=(df['X_res'].str.replace(',','').str.findall(r'(\d+\.?\d+)').apply(lambda x: pd.Series(x).astype(int)).mean(1))
df["X_res"]=pd.to_numeric(df["X_res"])

In [1]:
df.info()

In [1]:
# Calculation of PPI from X_res,Y_res,Inches
df["PPI"]=(((df["X_res"]**2+df["Y_res"]**2)**(1/2))/df["Inches"]).astype(float)

In [1]:
# Update the ScreenResolution now this coloumn is numeric value
df["ScreenResolution"]=(df["X_res"]*df["Y_res"]).astype(float)

In [1]:
# Object value convert to numeric values
df["Ram"] = df["Ram"].astype(int)
df["Weight"] = df["Weight"].astype(float)

In [1]:
# Drop unused columns from the Dataframe 
df=df.drop(['laptop_ID','first','second','Layer1HDD','Layer1SSD','Layer1Hybrid','Layer1Flash_Storage','Layer2HDD','Layer2SSD','Layer2Hybrid','Layer2Flash_Storage','Total_Memory'],axis=1)


In [1]:
df.info()

In [1]:
df.head(10)

# Preparation of Model Data and Scaling of Data

In [1]:
X=df.drop(['Price_euros'],axis=1)

In [1]:
# Log transform of y values
y=np.log(df['Price_euros'].values)

In [1]:
X=pd.concat([X,pd.get_dummies(X)],axis=1)
print(X.info())

In [1]:
X_=X.select_dtypes(exclude=['object'])

In [1]:
# Cleaning duplicate columns ..
X_ = X_.loc[:,~X_.columns.duplicated()]

In [1]:
# Normalization of Features
X=(X_-np.min(X_))/(np.max(X_)-np.min(X_))

In [1]:
print(X.info())

# Modelling

## LINEAR REGRESSION

In [1]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)


In [1]:
from sklearn.metrics import mean_squared_error
# Let's see what is the prediction error of our model.


y_pred_lr = lin_reg.predict(X)
lin_mse = mean_squared_error(y, y_pred_lr)
lin_rmse = np.sqrt(lin_mse)
print("Linear Regression MSE: ",lin_mse)
print("Linear Regression RMSE: ",lin_rmse)


In [1]:
# Let's see how accurate is our model.
from sklearn import metrics
accuracy_lin = metrics.r2_score(y, y_pred_lr)
print("Linear Regression r2: ",accuracy_lin)


In [1]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(y, y_pred_lr)
print("Linear Regression MAE: ",lin_mae)

## DECISION TREE

In [1]:
from sklearn.tree import DecisionTreeRegressor

tree_reg=DecisionTreeRegressor()
tree_reg.fit(X,y)


In [1]:
# Let's see what is the prediction error of our model.

y_pred_dt=tree_reg.predict(X)

dt_mse = mean_squared_error(y, y_pred_dt)
dt_rmse = np.sqrt(dt_mse)
print("Decision Tree Regression MSE: ",dt_mse)
print("Decision Tree Regression RMSE: ",dt_rmse)



In [1]:
# Let's see how accurate is our model.

accuracy_dt = metrics.r2_score(y, y_pred_dt)
print("Decision Tree Regression r2: ",accuracy_dt)

In [1]:
dt_mae = mean_absolute_error(y, y_pred_dt)
print("Decision Tree Regression MAE: ",dt_mae)

## RANDOM FOREST REGRESSION

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=42)

rf=RandomForestRegressor(n_estimators=100,random_state=42)

rf.fit(x_train,y_train)

In [1]:
# Let's see what is the prediction error of our model.

y_pred_rf=rf.predict(x_test)

rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_rmse = np.sqrt(rf_mse)

print("Random Forest Regression MSE: ",rf_mse)
print("Random Forest Regression RMSE: ",rf_rmse)

In [1]:
# Let's see how accurate is our model.

accuracy_rf=metrics.r2_score(y_test,y_pred_rf)

print("Random Forest Regression r2: ",accuracy_rf)

In [1]:
rf_mae = mean_absolute_error(y_test, y_pred_rf)
print("Random Forest Regression  MAE: ",rf_mae)